In [37]:
from IPython.display import HTML, display
from copy import deepcopy
import random
import os
import openai
import time
import sys
import ast
import json
from tqdm.auto import tqdm

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from getpass import getpass
import os
import pandas as pd

# import pdfkit
import html
from copy import deepcopy

In [23]:
# setting up an OpenAI template on the run
OPENAI_API_KEY = getpass()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [38]:
model = ChatOpenAI(model="gpt-4-0613")
model.temperature = 0.8

# conv_model = ChatOpenAI(model="gpt-4-0613")
# conv_model.temperature = 0.8

#### OS more tasks

In [105]:
import re
from tqdm.auto import tqdm 
from datasets import load_dataset
raw_dataset = load_dataset("THUDM/AgentInstruct", split="os")

task_list = []
for turn in raw_dataset:
    task_list.append(turn['conversations'][6]['value'].split("My problem is:\n\n")[-1])

Found cached dataset parquet (C:/Users/harsh/.cache/huggingface/datasets/THUDM___parquet/THUDM--AgentInstruct-ec062f298171fe56/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [6]:
system_prompt = \
"""You are given 5 examples of a task a user can perform in linux(ubuntu) operating system using bash commands. You have to make 3 more new and comprehensive tasks. Don't give the solution but only generate the task. 
NOTE: 
1) Be diverse and realistic while innovating the tasks. 
2) While creating the tasks you have the freedom to assume the state of the linux OS with files, folders and other features present. """

more_task =[]
for _ in tqdm(range(25)):
    sample_tasks = random.sample(task_list, 5)
    user_message_str = ""
    for i, (t) in enumerate(sample_tasks):
        user_message_str += f"Task{i+1}: {t.strip()}\n\n"
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_message_str.strip()),]
    response = model.predict_messages(messages)
    
    print(response.content)
    print('-'*50)
    
    new_tasks = [re.sub(r"Task\d+: ", "", t) for t in response.content.split("\n\n")]
    more_task.extend(new_tasks)

  0%|          | 0/25 [00:00<?, ?it/s]

NameError: name 'task_list' is not defined

#### Operating System Environment

In [40]:
environment_prompt_template = '''
Please take on the role of a linux(ubuntu) environment. I will send bash code and commands and you will respond with the output that is as close to the output of the linux environment as possible.
Limit the response to around 250 characters and note that the output has been truncated with the message  "[truncated because the output is too long]".  Do not repeat the command provided. 
Some examples of input bash commands and your outputs are as follows:

## Example 1
Input command: 

```bash
journalctl --list-boots
```

The output of the OS: 

journalctl: command not found\r\n\x1b]0;root@73ae112ca224: /\x07root@73ae112ca224:/#


## Example 2
Input command: 

```bash
ls *.txt | grep -i "^[aeiou]" | wc -l
```

The output of the OS: 

6\r\n\x1b[?2004h\x1b]0;root@b363de2463eb: /\x07root@b363de2463eb:/# \x1b[K


## Example 3
Input command: 

```bash
du -sh ~/*
```

The output of the OS: 

101M\t/root/demo1\r\n76M\t/root/demo2\r\n81M\t/root/demo3\r\n\x1b]0;root@3af35024d25d: /\x07root@3af35024d25d:/#


Input command:
```bash
mkdir ~/Documents/Project1
```


Input command:
```bash
{}
```
'''

# agent_prompt_template = '''
# Interact with a software environment to solve a task. Imagine you are an intelligent agent working for a user and your target is to perform actions to complete the task goal. At the beginning of your interactions, you will be given a detailed description of the current environment and your goal to accomplish. For each of your turns, you will be given a list of actions which you can choose one to perform in this turn. You should provide two parts of your response: "THOUGHT" and "ACTION". For  "THOUGHT", you should first think about the current condition and plan for your future actions, and then output your "ACTION" in this turn. Your output must strictly follow this format:"THOUGHT: your thoughts.
#  ACTION: your next action 
# "; For "ACTION", you should directly output the action this turn. Your output must strictly follow this format:"ACTION: your next action
# ". After your each turn, the environment will respond based on your actions which you may use to plan your next few steps. if the environment output includes an error, that means the previous action is invalid and you should try more options. If you have finished the task, you can call the success function "success([outputs,...])" with any final outputs.
#  Reminder: 
# 1. the action must follow any formats requested
# 2. Think when necessary, try to act directly more in the process.
# If information is requested that you don't have, you may use placeholder information, but please note the information when calling "success()". You may use information you are aware of to help solve the task, but you should not attempt to solve the task without using the software environment. 

# Software Environment: {}
# Your Task: {}
# '''

In [4]:
agent_prompt_template = \
'''
You are an assistant that will act like a person, I'will play the role of linux(ubuntu) operating system. Your goal is to implement the operations required by me or answer to the question proposed by me.
For each of your turn, you should first think what you should do, and then take exact one of the three actions: "bash", "finish" or "answer".

1. If you think you should execute some bash code, take bash action, and you should print like this:

Think: put your thought here.
Act: bash
```bash
# put your bash code here
```

2. If you think you have finished the task, take finish action, and you should print like this:

Think: put your thought here.
Act: finish

3. If you think you have got the answer to the question, take answer action, and you should print like this:

Think: put your thought here.
Act: answer(Your answer to the question should be put in this pair of parentheses)

If the output is too long, I will truncate it. The truncated output is not complete. You have to deal with the truncating problem by yourself. Attention, your bash code should not contain any input operation. Once again, you should take only exact one of the three actions in each turn.

Now my problem is:
{}
'''

In [5]:
environment_information_EV_prompt = "The environment is a linux(ubuntu) installation."
software_environment = "You are interacting with a linux(ubuntu) environment. You will use bash code and commands to accomplish your task."
agent_task = "tell me how many files are in the directory '/etc'?"

In [6]:
agent_prompt = agent_prompt_template.format(software_environment, agent_task)


agent_messages = [
    HumanMessage(content=agent_prompt),
]

In [7]:
agent_response = model.predict_messages(agent_messages)

In [8]:
agent_response.content

"THOUGHT: To find out the number of files in a directory in Linux, we can use the 'ls' command to list the files and then pipe that to the 'wc' command to count them. We should use `ls -l /etc | wc -l` to get the number of files. \n\nACTION: ls -l /etc | wc -l"

In [9]:
agent_result = agent_response.content.split("ACTION:")[1].strip()

In [10]:
agent_result

'ls -l /etc | wc -l'

In [11]:
environment_prompt = environment_prompt_template.format(environment_information_EV_prompt, agent_result)

environment_messages = [
    HumanMessage(content=environment_prompt)
]

In [12]:
environment_result = model.predict_messages(environment_messages)

In [13]:
environment_result.content

'152'

In [24]:
agent_messages.append(HumanMessage(content=environment_result.content))

In [15]:
num_turns = 10

for i in range(num_turns):
    agent_response = model.predict_messages(agent_messages)
    print(agent_response.content)
    agent_result = agent_response.content.split("ACTION:")[1].strip()
    environment_messages.append(HumanMessage(content=agent_result))
    agent_messages.append(agent_response)
    if "success(" in agent_result:
        break
    environment_result = model.predict_messages(environment_messages)
    environment_messages.append(environment_result)
    print(environment_result.content)
    agent_messages.append(HumanMessage(content=environment_result.content))

THOUGHT: The user has asked for the number of files in the '/etc' directory. I can get this information by using the 'ls' command in the Linux shell which lists directory contents. A simple 'ls /etc | wc -l' will give the number of files and directories within '/etc'. But, this counts directories as well, I need to specifically count the files. I can use 'find' command with '-type f' option which ensures only files are counted.

ACTION: find /etc -type f | wc -l
1154
THOUGHT: I have received the output from the previous command which indicates the number of files in the '/etc' directory. The output value is 1154, which means there are 1154 files in the '/etc' directory.

ACTION: success(1154)


#### Get tasks from AgentInstruct data

In [41]:
from datasets import load_dataset
raw_dataset = load_dataset("THUDM/AgentInstruct", split="os")
dataset = []
for ds in raw_dataset:
    dataset.append(ds)
df = pd.DataFrame(dataset)

Found cached dataset parquet (C:/Users/harsh/.cache/huggingface/datasets/THUDM___parquet/THUDM--AgentInstruct-ec062f298171fe56/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [42]:
df.iloc[2]["conversations"][6]['value'].split("is:")[1].strip()

'Count how many files or directories in the current directory have the read permission for the user.'

In [43]:
len(df)

195

In [44]:
def process(data):
    return data['conversations'][6]['value'].split("is:")[1].strip()

In [45]:
tasks = df.apply(process, axis=1).tolist()

In [46]:
from os_more_tasks import os_more_tasks
tasks = os_more_tasks

In [47]:
len(tasks)

226

#### Multiple conversations

In [49]:
import re
agent_init_msgs = []
for msg in ds["conversations"][:6]:
    if msg['from'] == "human":
        agent_init_msgs.append(HumanMessage(content=msg['value']))
    else:
        agent_init_msgs.append(AIMessage(content=msg['value']))
        

In [50]:
num_turns = 10

all_agents = []
all_environments = []

for task in tqdm(tasks): 
    # agent_prompt = agent_prompt_template.format(software_environment, x)
    agent_messages = agent_init_msgs + [
        HumanMessage(content=f'Now, I will start a new problem in a new OS. My problem is:\n\n{task}'),
    ]
    agent_response = model.predict_messages(agent_messages)
    print(agent_response.content)
    agent_messages.append(agent_response)
    first_bash_block = re.search(r"```bash\n(.*?)\n```", agent_response.content, re.DOTALL)
    if first_bash_block:
        bash_code = first_bash_block.group(1).strip()
    else:
        bash_code = ""
    environment_prompt = environment_prompt_template.format(bash_code)
    environment_messages = [
        HumanMessage(content=environment_prompt)
    ]
    environment_result = model.predict_messages(environment_messages)
    environment_result.content = environment_result.content.strip()
    environment_messages.append(environment_result)
    agent_messages.append(HumanMessage(content=environment_result.content))
    print(environment_result.content)
    SAVE = True
    for i in range(num_turns):
        try:
            agent_response = model.predict_messages(agent_messages)
            print(agent_response.content)
            first_bash_block = re.search(r"```bash\n(.*?)\n```", agent_response.content, re.DOTALL)
            if first_bash_block:
                bash_code = first_bash_block.group(1).strip()
            else:
                bash_code = ""
            environment_messages.append(HumanMessage(content=bash_code))
            agent_messages.append(agent_response)
            if "finish" in agent_response.content or "answer(" in agent_response.content:
                break
            environment_result = model.predict_messages(environment_messages)
            environment_messages.append(environment_result)
            print(environment_result.content)
            agent_messages.append(HumanMessage(content=environment_result.content))
        except:
            print("#################BadRequestError##################")
            SAVE = False
            break
    if SAVE:
        all_agents.append(deepcopy(agent_messages))
        all_environments.append(deepcopy(environment_messages))

  0%|          | 0/226 [00:00<?, ?it/s]

Think: First, I need to create the 'Project1' directory inside the '~/Documents' directory. 

Act: bash

```bash
mkdir ~/Documents/Project1
```
The output of the OS:

mkdir: cannot create directory ‘/root/Documents/Project1’: File exists
]0;root@3af35024d25d: /root@3af35024d25d:/#
Think: The directory 'Project1' already exists. So, I don't need to create it. I will now navigate to 'Project1' and create the files.

Act: bash

```bash
cd ~/Documents/Project1 && touch file1.txt file2.txt file3.txt
```
The output of the OS:

]0;root@3af35024d25d: /root/Documents/Project1root@3af35024d25d:~/Documents/Project1#
Think: Based on the output, I've successfully navigated to the 'Project1' directory and created the files 'file1.txt', 'file2.txt', and 'file3.txt'.

Act: finish
Think: The script should be put in the /etc/cron.hourly directory so that it can be run every hour. The script will use the `df` command to check disk usage and `mail` command to send email if disk usage is more than 90%.

In [51]:
print(len(all_agents), len(all_environments))

226 226


In [52]:
conversations_ = []
for chat in all_agents:
    conversation_ = []
    for side in chat:
        if side.type == "human":
            side_type = "human"
        else:
            side_type = "gpt"
        conversation_.append({"from": side_type, "value":side.content})
    conversations_.append(conversation_ )

In [53]:
with open('./synthetic-0/os-interactions-syn_env_syn_chat-2-raw.jsonl', 'w') as file:
    for conversation in conversations_:
        json_str = json.dumps(conversation)
        file.write(json_str + "\n")

### fs whole conversation

In [1]:
from langchain.chat_models import ChatOpenAI
from getpass import getpass
from langchain.schema import HumanMessage, AIMessage, SystemMessage
import os
import json

In [2]:
# setting up an OpenAI template on the run
OPENAI_API_KEY = getpass()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [3]:
model = ChatOpenAI(model="gpt-4")
model.temperature = 0.8

In [4]:
from datasets import load_dataset
all_data = []
dataset = load_dataset("THUDM/AgentInstruct", split="os")
for ds in dataset:
    all_data.append(ds)

Found cached dataset parquet (C:/Users/harsh/.cache/huggingface/datasets/THUDM___parquet/THUDM--AgentInstruct-ec062f298171fe56/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [5]:
def convert_to_convs(s):
    all_convs = []
    s = s.split("## USER: Now, I will start a new problem in a new OS. My problem is:")
    s = [ss for ss in s if ss.strip()!="" ]
    s = ["## USER: Now, I will start a new problem in a new OS. My problem is:"+ss for ss in s]
    for ss in s:
        _dict = {"conversations": []}
        for sss in ss.split("## USER:")[1:]:
            user_conv, agent_conv = sss.split('## AGENT:')[0].strip(), sss.split('## AGENT:')[1].strip()
            _dict["conversations"] += [{"from": "human", "value": user_conv, "loss": False}, {"from": "gpt", "value": agent_conv, "loss": True}]
        all_convs.append(_dict)
    return all_convs

In [6]:
from tqdm.auto import tqdm 
import random
num_turns = 1000
init_prompt = \
"""
Given is a conversations between an AI agent and a user. The user asks the agent to solve a task and in return the agent suggests the solution to that task. The agent produces bash commands which the user runs in his linux operating system and returns the output to the agent. You have to generate some more conversations following the previous. Pls be innovative in creating the task for the agent. 
"""
init_task_gen_prompt = \
"""
Given are some tasks which can be performed in linux operating system. These tasks are given to an AI agent which performs it in a real Linux OS environment. Can you generate one more such task which is different from the previous given ones?
For the task generation, follow the format of anyone of the given tasks.
"""
all_convs = []
all_tasks = []
for _ in tqdm(range(num_turns)):
    random.shuffle(all_data)
    prompt = ""
    for dd in all_data[0]['conversations'][:6]:
        _from = "USER" if dd['from']=="human" else "AGENT"
        prompt+=f"\n\n## {_from}: {dd['value']}"
        if "Now, I will start a new problem in a new OS. My problem is:\n\n" in dd["value"]:
            all_tasks.append(dd["value"].split("Now, I will start a new problem in a new OS. My problem is:\n\n")[-1])
    for d in all_data[1:7]:
        for dd in d["conversations"][6:]:
            _from = "USER" if dd['from']=="human" else "AGENT"
            prompt += f"\n\n## {_from}: {dd['value']}"
            if "Now, I will start a new problem in a new OS. My problem is:\n\n" in dd["value"]:
                all_tasks.append(dd["value"].split("Now, I will start a new problem in a new OS. My problem is:\n\n")[-1])
    task_messages =[
        SystemMessage(content=init_task_gen_prompt),
        HumanMessage(content="\n\n".join(all_tasks[-100:]))
    ]
    try:
        task_response = model.predict_messages(task_messages)
        all_tasks.append(task_response.content)
        task = task_response.content.strip()
        messages = [
            SystemMessage(content=init_prompt),
            HumanMessage(content=prompt+f"\n\n## USER: Now, I will start a new problem in a new OS. My problem is:\n\n{task}")
        ]   
        response = model.predict_messages(messages)
        convs = convert_to_convs(f"\n\n## USER: Now, I will start a new problem in a new OS. My problem is:\n\n{task}" + response.content)
        all_convs.extend(convs)    
        messages.append(response)
    except:
        print("Not the assumed format!!") 
        pass
    try:
        task_response = model.predict_messages(task_messages)
        all_tasks.append(task_response.content)
        task = task_response.content.strip()
        messages = [
            SystemMessage(content=init_prompt),
            HumanMessage(content=prompt+f"\n\n## USER: Now, I will start a new problem in a new OS. My problem is:\n\n{task}")
        ]   
        response = model.predict_messages(messages)
        convs = convert_to_convs(f"\n\n## USER: Now, I will start a new problem in a new OS. My problem is:\n\n{task}" +response.content)
        all_convs.extend(convs)    
        messages.append(response)
    except:
        print("Not the assumed format!!") 
        pass    
    try:
        task_response = model.predict_messages(task_messages)
        all_tasks.append(task_response.content)
        task = task_response.content.strip()
        messages = [
            SystemMessage(content=init_prompt),
            HumanMessage(content=prompt+f"\n\n## USER: Now, I will start a new problem in a new OS. My problem is:\n\n{task}")
        ]   
        response = model.predict_messages(messages)
        convs = convert_to_convs(f"\n\n## USER: Now, I will start a new problem in a new OS. My problem is:\n\n{task}" + response.content)
        all_convs.extend(convs)    
        messages.append(response)
    except:
        print("Not the assumed format!!") 
        pass

    with open('./synthetic-3/os_syn-datasyntype_2-2.json', 'w') as file:
        json.dump(all_convs, file)
        # for d in all_convs:
        #     json.dump(d, file)
        #     file.write('\n')
    # break

  0%|          | 0/1000 [00:00<?, ?it/s]

In [12]:
import json
from datasets import load_dataset
with open("./synthetic-3/os_syn-datasyntype_2.json", "r") as file:
    data = json.load(file)
    
agentinstruct_data = load_dataset("THUDM/AgentInstruct", split="os")

Found cached dataset parquet (C:/Users/harsh/.cache/huggingface/datasets/THUDM___parquet/THUDM--AgentInstruct-ec062f298171fe56/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [13]:
new_data = []
for ds in data:
    _ds = {"conversations": agentinstruct_data[0]["conversations"][:7] + ds["conversations"]} 
    new_data.append(_ds)

In [15]:
with open('./synthetic-3/os_syn-datasyntype_2.jsonl', 'w') as file:
    for d in new_data:
        json.dump(d, file)
        file.write('\n')